In [10]:
import sys
sys.path.append('../')

In [1]:
# %%capture
%run common.ipynb
# importnb allows us to load the code from other notebooks
# from importnb import Notebook
# with Notebook():
#     import OfficeAutomation.common as common

In [12]:
import crestdsl.model as crest
import crestdsl.model.api as api
import crestdsl.simulation as crestsim
import crestdsl.ui as ui

In [16]:
class Splitter(crest.Entity, ElectricalDevice):
    normal = current = crest.State()
    no_power = crest.State()
    
    
    req_sum = crest.Local(res_electricity, value=0)
    req_sum_to_req = crest.Influence(source=req_sum, target="req_electricity_out")
    
    # surcharge 
    surcharge = crest.Output(resource=res_bool, value=False)
    @crest.update(state=normal, target=surcharge)
    def update_surcharge_normal(self, dt):
        return False
    
    @crest.update(state=no_power, target=surcharge)
    def update_surcharge_no_power(self, dt):
        return True
    
    
    @crest.transition(source=normal, target=no_power)
    def to_no_power(self):
        return self.electricity_in.value < self.req_sum.value

    @crest.transition(source=no_power, target=normal)
    def to_normal(self):
        return self.electricity_in.value >= self.req_sum.value

    def connect_devices(self, devices):
        for dev in devices:
            api.add(self, f"elec_{dev._name}", crest.Output(res_electricity, value=0))
            api.add(self, f"req_{dev._name}", crest.Input(res_electricity, value=0))
            
            dev_after_init = getattr(self._parent, dev._name)
            api.add(self._parent, f"connect_elec_{self._name}_to_{dev._name}", crest.Influence(source=f"{self._name}.elec_{dev._name}", target=dev_after_init.electricity_in))
            api.add(self._parent, f"connect_{dev._name}_to_req_{self._name}", crest.Influence(source=dev_after_init.req_electricity_out, target=f"{self._name}.req_{dev._name}"))
            api.add(self, f"update_{dev._name}_elec_normal", crest.Update(state=self.normal, target=f"elec_{dev._name}", function=f"(lambda self, dt: self.req_{dev._name}.value)"))
            api.add(self, f"update_{dev._name}_elec_nopower", crest.Update(state=self.no_power, target=f"elec_{dev._name}", function=f"(lambda self, dt: 0)"))
        
        if len(devices) > 0:
            sum_of_reqs_src = " + ".join([f"self.req_{dev._name}.value" for dev in devices])
            update_req_src = f"(lambda self, dt: {sum_of_reqs_src} )"
            self.update_req = crest.Update(state=self.normal, target=self.req_sum, function=update_req_src)
            self.update_req_nopower = crest.Update(state=self.no_power, target=self.req_sum, function=update_req_src)


# ui.plot(Splitter())